In [3]:
import pandas as pd

# 加载数据
data = pd.read_csv('../../autodl-tmp/DataGame/Data_all_filter.csv')

# 准备交互数据
interactions = data[['user_id_index', 'parent_asin_index', 'unix_timestamp']].copy()
interactions['rating'] = 1  # 假设所有交互都有评分1
interactions['user_id_index'] = interactions['user_id_index'].astype(int)
interactions['parent_asin_index'] = interactions['parent_asin_index'].astype(int)
interactions['unix_timestamp'] = interactions['unix_timestamp'].astype(float)
interactions['rating'] = interactions['rating'].astype(float)
interactions['user_id_index'] = interactions['user_id_index'].astype('category').cat.codes
interactions['parent_asin_index'] = interactions['parent_asin_index'].astype('category').cat.codes
interactions.columns = ['user_id:token', 'item_id:token', 'timestamp:float', 'rating:float']

# 按 user_id:token 分组，并按时间排序
interactions = interactions.sort_values(by=['user_id:token', 'timestamp:float'])
grouped = interactions.groupby('user_id:token')

# 初始化数据
train_data = []
valid_data = []
test_data = []

# 遍历每个用户的交互数据
for user_id, group in grouped:
    if len(group) < 3:  # 如果用户交互少于 3 次，跳过
        continue
    # 提取用户的 item_id 列
    item_list = group['item_id:token'].tolist()

    # 分割数据
    train_items = item_list[:-2]  # 除去最后两项作为训练集
    valid_items = item_list[:-1]  # 除去最后一项作为验证集
    test_items = item_list  # 全部作为测试集

    # 添加训练集
    if len(train_items) > 0:
        train_data.append({
            'user_id:token': user_id,
            'item_id_list:token_seq': train_items,
            'item_id:token': item_list[-2]  # 训练集目标物品是倒数第二个
        })

    # 添加验证集
    valid_data.append({
        'user_id:token': user_id,
        'item_id_list:token_seq': valid_items,
        'item_id:token': item_list[-1]  # 验证集目标物品是最后一个
    })

    # 添加测试集
    test_data.append({
        'user_id:token': user_id,
        'item_id_list:token_seq': test_items[:-1],
        'item_id:token': item_list[-1]  # 测试集目标物品也是最后一个
    })

# 转换为 DataFrame
train_df = pd.DataFrame(valid_data)
valid_df = pd.DataFrame(valid_data)
test_df = pd.DataFrame(test_data)

print("Train, valid, and test splits saved successfully!")


Train, valid, and test splits saved successfully!


In [4]:
# data = pd.read_csv('../autodl-tmp/Data4Mul/Data_all_filter.csv') # Review_MultiGroup
#  data = pd.read_csv('../autodl-tmp/DataMovie/Data_all_filter.csv')
data = pd.read_csv('../../autodl-tmp/DataGame/Data_all_filter.csv')

interactions = data[['user_id_index', 'parent_asin_index', 'unix_timestamp']].copy()
interactions['rating'] = 1  # 假设所有交互都有评分1
interactions['user_id_index'] = interactions['user_id_index'].astype(int)
interactions['parent_asin_index'] = interactions['parent_asin_index'].astype(int)
interactions['unix_timestamp'] = interactions['unix_timestamp'].astype(float)
interactions['rating'] = interactions['rating'].astype(float)
interactions.columns = ['user_id:token', 'item_id:token', 'timestamp:float', 'rating:float']
interactions

,user_id:token,item_id:token,timestamp:float,rating:float
0,1,63963,1.563320e+09,1.0
1,1,60390,1.576420e+09,1.0
2,1,45365,1.578161e+09,1.0
3,1,59749,1.583088e+09,1.0
4,1,81643,1.593366e+09,1.0
...,...,...,...,...
1002375,114872,14089,1.384836e+09,1.0
1002376,114872,31862,1.464030e+09,1.0
1002377,114872,20884,1.464030e+09,1.0
1002378,114872,77945,1.638312e+09,1.0


In [5]:
items = data[['parent_asin', 'parent_asin_index']].drop_duplicates()

In [6]:
items

,parent_asin,parent_asin_index
0,B082R1RGZF,63963
1,B07SNN8GV5,60390
2,B01GY35T4S,45365
3,B07QX99XJJ,59749
4,B0BW17W9GM,81643
...,...,...
1002068,B08PCJVWLG,69337
1002180,B082PM9BSR,63935
1002181,B01N0K7IBT,47695
1002313,B00ES5BI0Y,33879


In [7]:
items.nunique()

parent_asin          82834
parent_asin_index    82834
dtype: int64

In [8]:
data.duplicated(subset=['user_id', 'parent_asin', 'unix_timestamp']).sum()

0

In [9]:
interactions.groupby(['user_id:token']).count()

,item_id:token,timestamp:float,rating:float
user_id:token,,,
1,7,7,7
2,12,12,12
3,10,10,10
4,12,12,12
5,5,5,5
...,...,...,...
114868,6,6,6
114869,8,8,8
114870,6,6,6


In [10]:
interactions.to_csv('./dataset/mydata/mydata.inter', sep='\t', index=False)

In [11]:
test_df

,user_id:token,item_id_list:token_seq,item_id:token
0,0,"[63962, 60389, 45364, 59748, 81642, 60479]",80279
1,1,"[26812, 21640, 30524, 26076, 33998, 48035, 462...",81335
2,2,"[5604, 8912, 9710, 19335, 6764, 10907, 14201, ...",26967
3,3,"[25041, 16695, 21996, 8259, 13752, 22423, 1799...",19681
4,4,"[37183, 57131, 37184, 37182]",26967
...,...,...,...
114867,114867,"[25233, 58656, 44303, 78908, 74344]",82173
114868,114868,"[14860, 28652, 41419, 29852, 14088, 40284, 50546]",80777
114869,114869,"[45707, 40839, 37332, 55027, 50938]",55247
114870,114870,"[33569, 23612, 28885, 21082, 26814, 38071, 807...",50938


In [12]:
train_df.to_csv('./dataset/mydata/mydata.train.inter', sep='\t', index=False)
valid_df.to_csv('./dataset/mydata/mydata.valid.inter', sep='\t', index=False)
test_df.to_csv('./dataset/mydata/mydata.test.inter', sep='\t', index=False)


In [13]:
metadata = pd.read_csv('../../autodl-tmp/DataGame/Item_all_filtered.csv')
metadata.head()

,parent_asin_index,parent_asin,title,categories
0,1,0078764343,Medal Of Honor: Warfighter Limited Edition (Xb...,Video Games;Legacy Systems;Xbox Systems;Xbox 3...
1,2,0152049495,Dogzilla,Video Games;Legacy Systems;Nintendo Systems
2,3,0156031191,Winter's Tale,Video Games;PC;Games
3,4,0310493099,It: How Churches and Leaders Can Get It and Ke...,Video Games;PC;Games
4,5,0316299936,Minions: Long Live King Bob!,Video Games;PC;Games


In [14]:
meta_filter = metadata[['parent_asin_index','title', 'categories']]
meta_filter['categories'] = (
    meta_filter['categories']
    .str.replace('</span>', '', regex=False)  # 删除 </span>
    .str.replace('[', '', regex=False)         # 删除 [
    .str.replace(']', '', regex=False)         # 删除 ]
    .str.replace("'", '', regex=False)         # 删除 ''
    .str.replace(",", '', regex=False)
    .str.strip()                               # 去掉前后空格
)
meta_filter.columns = ['item_id:token', 'title:token_seq', 'genre:token_seq']
meta_filter

C:\Users\iwuji\AppData\Local\Temp\ipykernel_12864\1332549472.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_filter['categories'] = (


,item_id:token,title:token_seq,genre:token_seq
0,1,Medal Of Honor: Warfighter Limited Edition (Xb...,Video Games;Legacy Systems;Xbox Systems;Xbox 3...
1,2,Dogzilla,Video Games;Legacy Systems;Nintendo Systems
2,3,Winter's Tale,Video Games;PC;Games
3,4,It: How Churches and Leaders Can Get It and Ke...,Video Games;PC;Games
4,5,Minions: Long Live King Bob!,Video Games;PC;Games
...,...,...,...
83062,83063,Dinosoo Wireless Controller for Xbox Series X/...,Video Games;Xbox One;Accessories;Controllers;G...
83063,83064,Cute Switch Carry Case for Nintendo Switch/Lit...,Video Games;Nintendo Switch;Accessories;Cases ...
83064,83065,HOZKAII Replacement for Playstation 4 Controll...,Video Games;PlayStation 4;Accessories;Cases & ...
83065,83066,"WolfLawS M6 RGB Gaming Mouse, 12800 DPI Wired ...",Video Games;PC;Accessories;Gaming Mice


In [15]:
meta_filter.to_csv('./dataset/mydata/mydata.item', sep='\t', index=False)